In [1]:
import json, os
import sys, tabulate


dataset_list = ["data_RAD","vqa_rad", "Slake1.0", "pvqa"]
split_list = ["train", "val", "test"]
for ds in dataset_list:
    for sp in split_list:
        split_file_path = f"/home/chunyl/azure_mount/hanoverdev/clwon/llava/eval/{ds}/{sp}.json"

        if os.path.exists(split_file_path):
            gt = json.load(open(split_file_path, 'r'))
            # gt

            # 'answer_type'
            gt_imgs = list(set([item['image'] for item in gt]))
            num_imgs = len(gt_imgs)

            gt_ids = [item['id'] for item in gt]
            num_ids = len(gt_ids)

            num_open, num_close = 0, 0
            for item in gt:
                if item['answer_type'] == 'OPEN':
                    num_open += 1
                elif item['answer_type'] == 'CLOSED': 
                    num_close += 1
                    
            

            print(f"{ds} {sp}: {num_imgs} {num_ids} {num_open} {num_close} ")


data_RAD train: 314 3064 1241 1823 
data_RAD test: 203 451 179 272 
vqa_rad train: 313 1797 770 1027 
vqa_rad test: 203 451 179 272 
Slake1.0 train: 450 4919 2976 1943 
Slake1.0 val: 96 1053 631 422 
Slake1.0 test: 96 1061 645 416 
pvqa train: 2599 19755 9949 9806 
pvqa val: 832 6279 3144 3135 
pvqa test: 858 6761 3370 3391 


In [2]:
gt[-1]

{'id': 300648006,
 'image': 'test/test_0648.jpg',
 'answer_type': 'OPEN',
 'question_type': 'where',
 'conversations': [{'from': 'human', 'value': 'Where is this?\n<image>'},
  {'from': 'gpt', 'value': 'urinary'}]}

In [3]:
pred_list_file = "/home/chunyl/research/project/2023/llava_biomed/LLaVA_v1/eval_results_med_datasets.jsonl"

pred_list = open(pred_list_file, "r")

pred_dict_list = []
for pred in pred_list:
    # print(pred)
    pred_dict = json.loads(pred)
    pred_dict_list.append(pred_dict)

    print(pred_dict["pred_file"])


/home/chunyl/azure_mount/chunyleu_output/llava-med/llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/zero-shot-on-fixed-data/pvqa/test-answer-file.jsonl
/home/chunyl/azure_mount/chunyleu_output/llava-med/llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/zero-shot-on-fixed-data/Slake1.0/test-answer-file.jsonl
/home/chunyl/azure_mount/chunyleu_output/llava-med/llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/zero-shot-on-fixed-data/data_RAD/test-answer-file.jsonl
/home/chunyl/azure_mount/chunyleu_output/llava-med/llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/finetune_to_downstream_results_fixed_data/data_RAD-9epoch/test-answer-file.jsonl
/home/chunyl/azure_mount/chunyleu_o

In [4]:

metric_list = ['exact match score', 'f1 score', 'precision', 'recall', 'open accuracy', 'yes/no accuracy'] 

def get_metrics(dataset='Slake1.0'):

    dict_eval_ds= {}
    for pred_dict in pred_dict_list:
        if pred_dict['dataset'] == dataset:

            string_data = pred_dict['results']
            # split the string into rows  
            rows = string_data.split("\n")  
            
            metric_dict = {}
            for r in rows[:-1]:
                for m in metric_list:
                    if m in r:
                        metric_dict[m] = float( r.split(m)[-1] )

            json_path = "/".join(pred_dict["pred_file"].split('/')[5:])
            print(json_path)
            print(metric_dict,"\n")

            dict_eval_ds[json_path] = metric_dict

    return dict_eval_ds







In [5]:
dict_eval_vqa_rad = get_metrics(dataset='vqa_rad')


llava-med/data/llava/models/med-pretrain-364m-v1-1epoch/eval/zero-shot/vqa_rad/test-answer-file.jsonl
{'exact match score': 1.80001, 'f1 score': 3.31813, 'precision': 1.97184, 'recall': 13.7606, 'open accuracy': 7.26257, 'yes/no accuracy': 14.3382} 

llava-med/data/llava/models/finetune_e2e_on_instruct-3epoch/finetune_to_downstream_results/vqa_rad-3epoch/test-answer-file.jsonl
{'exact match score': 14.3669, 'f1 score': 16.6764, 'precision': 14.3669, 'recall': 21.797, 'open accuracy': 16.7598, 'yes/no accuracy': 62.5} 

llava-med/data/llava/models/finetune_e2e_on_instruct-3epoch/finetune_to_downstream_results/vqa_rad-1epoch/test-answer-file.jsonl
{'exact match score': 13.5475, 'f1 score': 16.03, 'precision': 13.5475, 'recall': 21.5829, 'open accuracy': 16.7598, 'yes/no accuracy': 58.4559} 

llava-med/data/llava/models/finetune_e2e_on_instruct_cleaned-60k-3epoch/zero-shot-on-fixed-data/vqa_rad/test-answer-file.jsonl
{'exact match score': 3.75327, 'f1 score': 6.83915, 'precision': 4.22844

In [6]:
dict_eval_data_RAD = get_metrics(dataset='data_RAD')

llava-med/llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/zero-shot-on-fixed-data/data_RAD/test-answer-file.jsonl
{'exact match score': 4.80739, 'f1 score': 8.307, 'precision': 5.14287, 'recall': 31.656, 'open accuracy': 17.3184, 'yes/no accuracy': 61.3971} 

llava-med/llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/finetune_to_downstream_results_fixed_data/data_RAD-9epoch/test-answer-file.jsonl
{'exact match score': 42.7086, 'f1 score': 50.8654, 'precision': 42.9647, 'recall': 64.581, 'open accuracy': 60.3352, 'yes/no accuracy': 77.9412} 

llava-med/data/llava/models/finetune_e2e_on_instruct_cleaned-60k-3epoch/zero-shot-on-fixed-data/data_RAD/test-answer-file.jsonl
{'exact match score': 4.35804, 'f1 score': 7.77379, 'precision': 4.81828, 'recall': 30.6541, 'open accuracy': 18.4358, 'yes/no accuracy': 55.5147} 

llava-med/llav

In [7]:
dict_eval_slake = get_metrics(dataset='Slake1.0')

llava-med/llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/zero-shot-on-fixed-data/Slake1.0/test-answer-file.jsonl
{'exact match score': 3.82914, 'f1 score': 7.53261, 'precision': 4.38287, 'recall': 37.7117, 'open accuracy': 32.4031, 'yes/no accuracy': 49.7596} 

llava-med/llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/finetune_to_downstream_results_fixed_data/Slake1.0-9epoch/test-answer-file.jsonl
{'exact match score': 47.9101, 'f1 score': 61.0086, 'precision': 48.4837, 'recall': 84.9668, 'open accuracy': 79.6899, 'yes/no accuracy': 85.5769} 

llava-med/data/llava/models/finetune_e2e_on_instruct_cleaned-60k-3epoch/zero-shot-on-fixed-data/Slake1.0/test-answer-file.jsonl
{'exact match score': 3.27036, 'f1 score': 6.57152, 'precision': 3.78599, 'recall': 35.5258, 'open accuracy': 29.4574, 'yes/no accuracy': 53.8462} 

llava-med/

In [8]:
dict_eval_pvqa = get_metrics(dataset='pvqa')

llava-med/llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/zero-shot-on-fixed-data/pvqa/test-answer-file.jsonl
{'exact match score': 0.925277, 'f1 score': 2.31502, 'precision': 1.40977, 'recall': 11.3417, 'open accuracy': 5.01484, 'yes/no accuracy': 49.6314} 

llava-med/llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/finetune_to_downstream_results_fixed_data/pvqa-9epoch/test-answer-file.jsonl
{'exact match score': 22.3608, 'f1 score': 27.822, 'precision': 22.4283, 'recall': 38.8213, 'open accuracy': 35.2522, 'yes/no accuracy': 92.3916} 

llava-med/data/llava/models/finetune_e2e_on_instruct_cleaned-60k-3epoch/zero-shot-on-fixed-data/pvqa/test-answer-file.jsonl
{'exact match score': 0.801299, 'f1 score': 2.29458, 'precision': 1.36786, 'recall': 11.7571, 'open accuracy': 4.62908, 'yes/no accuracy': 53.1996} 

llava-med/llava-vicun

In [9]:
dict_eval_ds_list = [dict_eval_data_RAD, dict_eval_slake, dict_eval_pvqa]

exp_list = [
    "llava-vicuna-7b/med-train_e2e-instruct-data-run-from-llava/ablation/no-stage1-3epoch",
    "med-train_e2e-instruct-data-run-from-llava-p-3epoch/zero-shot-on-fixed-data",
    "llava-vicuna-7b-pretrain_cc3m_595k_1e-instruct_158k-3epoch/eval/zero-shot-on-fixed-data",
    "llava-vicuna-7b-pretrain_cc3m_595k_1e-instruct_158k-3epoch/finetune_to_downstream_results_fixed_data/*-1epoch",
    "llava-vicuna-7b-pretrain_cc3m_595k_1e-instruct_158k-3epoch/finetune_to_downstream_results_fixed_data/*-3epoch",
    "med-pretrain-364m-1epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-3epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct-3epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-3epoch/finetune_e2e_on_instruct_run2-3epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_postprocess_cleaned_10k_qa_44k_with_image-3epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_postprocess_im_cleaned_10k_qa_48k_with_image-3epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-3epoch/finetune_e2e_on_instruct-3epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-1epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-3epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_cleaned-60k-1epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_cleaned-60k-3epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-1epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-9epoch/zero-shot-on-fixed-data",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-1epoch/finetune_to_downstream_results_fixed_data/*-1epoch",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-1epoch/finetune_to_downstream_results_fixed_data/*-3epoch",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-3epoch/finetune_to_downstream_results_fixed_data/*-1epoch", 
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-3epoch/finetune_to_downstream_results_fixed_data/*-3epoch", 
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct-3epoch/finetune_to_downstream_results_fixed_data/*-1epoch",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct-3epoch/finetune_to_downstream_results_fixed_data/*-3epoch",
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/finetune_to_downstream_results_fixed_data/*-1epoch",   
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/finetune_to_downstream_results_fixed_data/*-3epoch",   
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/finetune_to_downstream_results_fixed_data/*-9epoch",   
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/finetune_to_downstream_results_fixed_data/*-12epoch",   
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/finetune_to_downstream_results_fixed_data/*-15epoch",  
    "med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/finetune_to_downstream_results_fixed_data/*-18epoch",  
    "med-pretrain-1epoch-BiomedCLIP/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/zero-shot-on-fixed-data",
    "med-pretrain-1epoch-BiomedCLIP/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/finetune_to_downstream_results_fixed_data/*-9epoch",
    "med-pretrain-1epoch-BiomedCLIP/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/finetune_to_downstream_results_fixed_data/*-15epoch",
    "llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/zero-shot-on-fixed-data",
    "llava-vicuna-13b/llava_med-llava-13b-v0-pretrain-364m-v1-1epoch_consolidate/finetune_e2e_on_instruct_caption_in_text_cleaned-60k/finetune_to_downstream_results_fixed_data"
]


exp_results_list = [] 
for exp in exp_list:

    exp_results = []
    for dict_eval_ds in dict_eval_ds_list:
        for key, metric_value in dict_eval_ds.items():
            if "*" in exp and exp.split("*")[0] in key and exp.split("*")[1]+"/test-answer-file" in key:
                # print(exp.split("*"), key)
                recall, accuracy = dict_eval_ds[key]['recall'], dict_eval_ds[key]['yes/no accuracy']
                exp_results.append(recall)
                exp_results.append(accuracy)
            elif exp in key:
                recall, accuracy = dict_eval_ds[key]['recall'], dict_eval_ds[key]['yes/no accuracy']
                exp_results.append(recall)
                exp_results.append(accuracy)

    exp_results_list.append(exp_results)

for exp, exp_results in zip(exp_list, exp_results_list):
    print(exp)

    res = ''
    for r in exp_results:
        res += f'{r:.2f} & '

    avg = sum(exp_results)/len(exp_results)
    res += f'{avg:.2f} '
    
    print(res)

llava-vicuna-7b/med-train_e2e-instruct-data-run-from-llava/ablation/no-stage1-3epoch
26.94 & 56.62 & 36.80 & 57.21 & 7.43 & 57.68 & 40.45 
med-train_e2e-instruct-data-run-from-llava-p-3epoch/zero-shot-on-fixed-data
27.02 & 58.46 & 34.30 & 57.93 & 8.38 & 57.39 & 40.58 
llava-vicuna-7b-pretrain_cc3m_595k_1e-instruct_158k-3epoch/eval/zero-shot-on-fixed-data
20.74 & 59.19 & 26.82 & 50.24 & 8.74 & 45.65 & 35.23 
llava-vicuna-7b-pretrain_cc3m_595k_1e-instruct_158k-3epoch/finetune_to_downstream_results_fixed_data/*-1epoch
29.96 & 52.21 & 69.03 & 46.63 & 26.92 & 81.78 & 51.09 
llava-vicuna-7b-pretrain_cc3m_595k_1e-instruct_158k-3epoch/finetune_to_downstream_results_fixed_data/*-3epoch
50.00 & 65.07 & 78.18 & 63.22 & 7.74 & 63.20 & 54.57 
med-pretrain-364m-1epoch/zero-shot-on-fixed-data
15.27 & 12.50 & 18.55 & 13.46 & 6.26 & 13.51 & 13.26 
med-pretrain-364m-3epoch/zero-shot-on-fixed-data
15.33 & 15.44 & 23.61 & 15.38 & 6.35 & 14.74 & 15.14 
med-pretrain-364m-1epoch/finetune_e2e_on_instruct-3epo

In [10]:
pred_list_file = "/home/chunyl/research/project/2023/llava_biomed/LLaVA_v1/eval_results_med_datasets_w_options.jsonl"

pred_list = open(pred_list_file, "r")

pred_dict_list = []
for pred in pred_list:
    # print(pred)
    pred_dict = json.loads(pred)
    pred_dict_list.append(pred_dict)

    print(pred_dict["pred_file"])

/home/chunyl/azure_mount/chunyleu_output/llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/zero-shot-on-data-w-options/data_RAD/test_w_options-answer-file.jsonl
/home/chunyl/azure_mount/chunyleu_output/llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/zero-shot-on-data-w-options/Slake1.0/test_w_options-answer-file.jsonl
/home/chunyl/azure_mount/chunyleu_output/llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/zero-shot-on-data-w-options/pvqa/test_w_options-answer-file.jsonl
/home/chunyl/azure_mount/chunyleu_output/llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-3epoch/zero-shot-on-data-w-options/data_RAD/test_w_options-answer-file.jsonl
/home/chunyl/azure_mount/chunyleu_output/llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-3epoch/zero-shot-on

In [8]:
dict_eval_pvqa = get_metrics(dataset='data_RAD')
dict_eval_pvqa = get_metrics(dataset='Slake1.0')
dict_eval_pvqa = get_metrics(dataset='pvqa')

llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch/zero-shot-on-data-w-options/data_RAD/test_w_options-answer-file.jsonl
{'exact match score': 3.74267, 'f1 score': 7.11678, 'precision': 4.39986, 'recall': 29.2483, 'open accuracy': 7.82123, 'yes/no accuracy': 83.871} 

llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-3epoch/zero-shot-on-data-w-options/data_RAD/test_w_options-answer-file.jsonl
{'exact match score': 3.34648, 'f1 score': 6.15806, 'precision': 3.75533, 'recall': 26.5786, 'open accuracy': 8.37989, 'yes/no accuracy': 82.0225} 

llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-3epoch/finetune_to_downstream_results_fixed_data/data_RAD-3epoch/test_w_options-answer-file.jsonl
{'exact match score': 22.2098, 'f1 score': 26.2827, 'precision': 22.2098, 'recall': 34.6648, 'open accuracy': 24.581, 'yes/no accuracy': 90.2703} 

llava-med/llava-vicuna-7b/med-pretrain-

In [50]:
dict_eval_vqa_rad
dict_eval_data_RAD

def get_evaluated_ckpts(dict_eval_vqa):
    ckpts_eval = []
    for k, v in dict_eval_vqa.items():
        ckpt_path = '/'.join(k.split('/')[:-2])
        ckpts_eval.append(ckpt_path)
    return ckpts_eval


ckpts_eval_vqa_rad = get_evaluated_ckpts(dict_eval_vqa_rad)
ckpts_eval_data_RAD = get_evaluated_ckpts(dict_eval_data_RAD)

ckpts_eval_vqa_rad = list(set(ckpts_eval_vqa_rad))
ckpts_eval_data_RAD = list(set(ckpts_eval_data_RAD))


In [58]:
ft_prefix = "finetune_to_downstream_results_fixed_data"
zs_prefix = "zero-shot-on-fixed-data"

ckpts_eval_vqa_rad_ft, ckpts_eval_vqa_rad_zs = [], []
for ckpt_path in ckpts_eval_vqa_rad:
    if ft_prefix in ckpt_path:
        ckpts_eval_vqa_rad_ft.append(os.path.dirname(ckpt_path))
    if zs_prefix in ckpt_path:
        ckpts_eval_vqa_rad_zs.append(os.path.dirname(ckpt_path))

ckpts_eval_data_RAD_ft, ckpts_eval_data_RAD_zs = [], []
for ckpt_path in ckpts_eval_data_RAD:
    if ft_prefix in ckpt_path:
        ckpts_eval_data_RAD_ft.append(os.path.dirname(ckpt_path))
    if zs_prefix in ckpt_path:
        ckpts_eval_data_RAD_zs.append(os.path.dirname(ckpt_path))

ckpts_eval_data_RAD_ft_difference = list(set(ckpts_eval_vqa_rad_ft) - set(ckpts_eval_data_RAD_ft)) 
ckpts_eval_data_RAD_zs_difference = list(set(ckpts_eval_vqa_rad_zs) - set(ckpts_eval_data_RAD_zs)) 


In [59]:
ckpts_eval_data_RAD_ft_difference

['llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-3epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-1epoch',
 'llava-med/llava-vicuna-7b/llava-vicuna-7b-pretrain_cc3m_595k_1e-instruct_158k-3epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct-3epoch']

In [60]:
ckpts_eval_data_RAD_zs_difference

['llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct_cleaned-60k-1epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct_cleaned-60k-3epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-3epoch/finetune_e2e_on_instruct-3epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-1epoch',
 'llava-med/data/llava/models/finetune_e2e_on_instruct_cleaned-60k-3epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-1epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct-3epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct_caption_in_text_cleaned-60k-3epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-1epoch/finetune_e2e_on_instruct-60k-3epoch',
 'llava-med/llava-vicuna-7b/med-pretrain-364m-3epoch',
 'llava-med/llava-vicuna-7b/ll